## What is the relationship between children's book texts and YouTube transcripts?  
To examine this: 
* Load Facebook AI's children's books corpus
* Extract transcripts from 5 YT videos

### Open FB's Book Corpus

In [1]:
! pip install --pre python-docx
! pip3 install --user lxml

    100% |████████████████████████████████| 5.3MB 3.5MB/s eta 0:00:01
    100% |████████████████████████████████| 8.7MB 2.4MB/s eta 0:00:01
  Running setup.py bdist_wheel for python-docx ... done
  Stored in directory: /Users/shawnramirez/Library/Caches/pip/wheels/2d/f7/de/d66fcac9876a765dfd2fbd64ff46e5f3322621800a3cc0130b
Successfully built python-docx
mkl-random 1.0.1 requires cython, which is not installed.
mkl-fft 1.0.0 requires cython, which is not installed.
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 8.7MB 112kB/s eta 0:00:01
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd

In [6]:
import os 
os.getcwd()

'/Users/shawnramirez/Google Drive/mypy/better screen time'

In [10]:
file = open('CBTest/data/cbt_train.txt', 'r')
books = file.read()

In [11]:
type(books)

str

In [14]:
books[0:1000]

'_BOOK_TITLE_ : Andrew_Lang___Prince_Prigio.txt.out\nCHAPTER I. -LCB- Chapter heading picture : p1.jpg -RCB- How the Fairies were not Invited to Court .\nOnce upon a time there reigned in Pantouflia a king and a queen .\nWith almost everything else to make them happy , they wanted one thing : they had no children .\nThis vexed the king even more than the queen , who was very clever and learned , and who had hated dolls when she was a child .\nHowever , she , too in spite of all the books she read and all the pictures she painted , would have been glad enough to be the mother of a little prince .\nThe king was anxious to consult the fairies , but the queen would not hear of such a thing .\nShe did not believe in fairies : she said that they had never existed ; and that she maintained , though The History of the Royal Family was full of chapters about nothing else .\nWell , at long and at last they had a little boy , who was generally regarded as the finest baby that had ever been seen .

In [15]:
len(books)

25742250

In [20]:
# split on line delimiter
split_as_list = books.split('\n')
split_as_list[:10]

['_BOOK_TITLE_ : Andrew_Lang___Prince_Prigio.txt.out',
 'CHAPTER I. -LCB- Chapter heading picture : p1.jpg -RCB- How the Fairies were not Invited to Court .',
 'Once upon a time there reigned in Pantouflia a king and a queen .',
 'With almost everything else to make them happy , they wanted one thing : they had no children .',
 'This vexed the king even more than the queen , who was very clever and learned , and who had hated dolls when she was a child .',
 'However , she , too in spite of all the books she read and all the pictures she painted , would have been glad enough to be the mother of a little prince .',
 'The king was anxious to consult the fairies , but the queen would not hear of such a thing .',
 'She did not believe in fairies : she said that they had never existed ; and that she maintained , though The History of the Royal Family was full of chapters about nothing else .',
 'Well , at long and at last they had a little boy , who was generally regarded as the finest baby 

### Load transcripts from YT

In [25]:
!pip install pysrt
!pip install cython

  Running setup.py bdist_wheel for srt ... done
  Stored in directory: /Users/shawnramirez/Library/Caches/pip/wheels/a3/17/9b/28b389ea9cf4983b5964b847e3c1cdd84b40bd152779940d73
Successfully built srt
mkl-random 1.0.1 requires cython, which is not installed.
mkl-fft 1.0.0 requires cython, which is not installed.


In [35]:
#https://pypi.org/project/pysrt/
import pysrt

In [107]:
# Concatenate the lines for each transcript in the directory
transcript_directory = os.fsencode('YTtranscripts/')

transcript_list = []

for file in os.listdir(transcript_directory):
    filename = os.fsdecode(file)
    if filename.endswith(".srt"): 
        filename = "YTtranscripts/" + filename
        subs = pysrt.open(filename)
        transtring = ''
        for i in range(len(subs)): 
            transtring += " " + subs[i].text
        transcript_list.append(transtring)
        print(file, filename)
        continue
    else:
        continue

b'CZ6_iGvtjds.srt' YTtranscripts/CZ6_iGvtjds.srt
b'9x4zQ9T2Zhw.srt' YTtranscripts/9x4zQ9T2Zhw.srt
b'BdDnT7jSAYA.srt' YTtranscripts/BdDnT7jSAYA.srt
b'j2WhBtmpqCA.srt' YTtranscripts/j2WhBtmpqCA.srt
b'fe5Jk7YT_JE.srt' YTtranscripts/fe5Jk7YT_JE.srt


In [110]:
print(transcript_list[4])

 Peanut peanut butter, and jelly Peanut peanut butter, and jelly First you take the peanuts and you crack 'em
You crack 'em You crack 'em, crack 'em, crack 'em Peanut peanut butter, and jelly Peanut peanut butter, and jelly Then you take the peanuts and you mash 'em
You mash 'em You mash 'em, mash 'em, mash 'em Peanut peanut butter, and jelly Peanut peanut butter, and jelly Then you take the bread and you spread it
You spread it You spread it, spread it, spread it Peanut peanut butter, and jelly Peanut peanut butter, and jelly Then you pick the grapes and you pick 'em
You pick 'em You pick 'em, pick 'em, pick 'em Peanut peanut butter, and jelly Peanut peanut butter, and jelly Then you take the grapes and you squish 'em
You squish 'em You squish 'em, squish 'em, squish 'em Peanut peanut butter, and jelly Peanut peanut butter, and jelly Then you take the bread and you spread it
You spread it You spread it, spread it, spread it Peanut peanut butter, and jelly Peanut peanut butter, and jel